In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
# Load the trained model
price_model = load_model("price_recommendation_model.keras")

C:\Users\Manju\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [3]:
# Load preprocessed dataset (used during training)
data = pd.read_csv('preprocessed_trade_data.csv')

In [12]:
# Function to prepare data for a specific commodity
def prepare_input_for_commodity(commodity_name, sequence_length=100):
    # Filter dataset for the given commodity
    commodity_data = data[data['commodity'] == commodity_name]
    if commodity_data.empty:
        raise ValueError(f"No data found for commodity: {commodity_name}")
    
    # Sort data by date to maintain temporal order
    commodity_data = commodity_data.sort_values(by='created_at')
    
    # Define features used during training
    features = ['min_price', 'max_price', 'commodity_arrivals', 'commodity_traded',
                'price_spread', 'price_ratio', 'demand_supply_ratio', 'month', 'day_of_week']
    
    # Ensure we have enough data for the sequence length
    if len(commodity_data) < sequence_length:
        raise ValueError(f"Not enough data to create sequences for {commodity_name}. Required: {sequence_length}, Found: {len(commodity_data)}")
    
    # Create input sequence from the most recent data
    sequence = commodity_data[features].iloc[-sequence_length:].values
    sequence = np.expand_dims(sequence, axis=0)  # Reshape to match LSTM input shape
    return sequence

In [13]:
# Function to predict price for a given commodity
def predict_price_for_commodity(commodity_name):
    try:
        # Prepare input sequence
        input_sequence = prepare_input_for_commodity(commodity_name)
        
        # Make prediction
        predicted_price = price_model.predict(input_sequence)
        predicted_price = predicted_price[0][0]  # Extract single value from prediction
        
        # Return the result
        return f"The recommended modal price for {commodity_name} is: ₹{predicted_price:.2f}"
    except ValueError as e:
        return str(e)

In [18]:
# Example usage
commodity_name = "COCONUT"  # Farmer-provided input
result = predict_price_for_commodity(commodity_name)
print(result)

No data found for commodity: COCONUT


In [21]:
import json
# Regenerate the mapping if not saved
original_data = pd.read_csv("yearly_trade_data.csv")  # Load original dataset
commodity_mapping = {name: idx for idx, name in enumerate(original_data['commodity'].unique())}

# Save the mapping for future use
with open("commodity_mapping.json", "w") as f:
    json.dump(commodity_mapping, f)


In [37]:
import numpy as np
import pandas as pd
import json

# Load preprocessed data
data = pd.read_csv("preprocessed_trade_data.csv")

# Load the commodity mapping
with open("commodity_mapping.json", "r") as f:
    commodity_mapping = json.load(f)

# Function to prepare input for a specific commodity
def prepare_input_for_commodity(commodity_name, sequence_length=30):
    # Normalize the commodity name and get its numeric code
    commodity_name = commodity_name.lower()
    commodity_numeric = 1
    
    if commodity_numeric is None:
        raise ValueError(f"No data found for commodity: {commodity_name}")
    
    # Filter dataset for the given commodity
    commodity_data = data[data['commodity'] == commodity_numeric]
    if commodity_data.empty:
        raise ValueError(f"No data found for commodity: {commodity_name}")
    
    # Sort data by date to maintain temporal order
    commodity_data = commodity_data.sort_values(by='price_ratio')
    
    # Define features used during training
    features = ['min_price', 'max_price', 'commodity_arrivals', 'commodity_traded',
                'price_spread', 'price_ratio', 'demand_supply_ratio', 'month', 'day_of_week']
    
    # Ensure we have enough data for the sequence length
    if len(commodity_data) < sequence_length:
        raise ValueError(f"Not enough data to create sequences for {commodity_name}. Required: {sequence_length}, Found: {len(commodity_data)}")
    
    # Create input sequence from the most recent data
    sequence = commodity_data[features].iloc[-sequence_length:].values
    sequence = np.expand_dims(sequence, axis=0)  # Reshape to match LSTM input shape
    return sequence

# Function to predict price for a given commodity
def predict_price_for_commodity(commodity_name):
    try:
        # Prepare input sequence
        input_sequence = prepare_input_for_commodity(commodity_name)
        
        # Make prediction
        predicted_price = price_model.predict(input_sequence)
        predicted_price = predicted_price[0][0]  # Extract single value from prediction
        
        # Return the result
        return f"The recommended modal price for {commodity_name.capitalize()} is: ₹{predicted_price:.2f}"
    except ValueError as e:
        return str(e)

# Example usage
commodity_name = "COCONUT"  # Farmer-provided input
result = predict_price_for_commodity(commodity_name)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
The recommended modal price for Coconut is: ₹-0.00


In [39]:
def predict_price(commodity_name):
    # Convert commodity to its numeric encoding
    commodity_name = commodity_name.lower()
    commodity_numeric = 1
    
    if commodity_numeric is None:
        raise ValueError(f"No data found for commodity: {commodity_name}")
    
    # Filter the dataset for the given commodity
    commodity_data = data[data['commodity'] == commodity_numeric]
    if commodity_data.empty:
        raise ValueError(f"No data found for commodity: {commodity_name}")
    
    # Prepare input sequence
    commodity_data = commodity_data.sort_values(by='month')  # Ensure chronological order
    features = ['min_price', 'max_price', 'commodity_arrivals', 'commodity_traded',
                'price_spread', 'price_ratio', 'demand_supply_ratio', 'month', 'day_of_week']
    
    sequence = commodity_data[features].iloc[-30:].values  # Last 30 days
    sequence = np.expand_dims(sequence, axis=0)  # Reshape for LSTM input
    
    # Make prediction
    predicted_price = model.predict(sequence)
    
    # Inverse transform the predicted price
    scaler = joblib.load('price_scaler.pkl')
    predicted_price = scaler.inverse_transform([[predicted_price[0][0]]])[0][0]
    
    return round(predicted_price, 2)

# Example usage
commodity = "coconut"
predicted_modal_price = predict_price(commodity)
print(f"The recommended modal price for {commodity.capitalize()} is: ₹{predicted_modal_price}")


NameError: name 'model' is not defined